# Project Summary
This project is a medical chatbot that uses Retrieval-Augmented Generation (RAG) to answer user queries based on content from "The Gale Encyclopedia of Medicine" PDF. It processes the PDF, indexes it for quick retrieval, and generates responses via an LLM. Built and tested on Kaggle, it's deployed as an interactive web app on Streamlit for easy user access.

### Tech Stack Used
- **Core Frameworks/Libraries**: Python, LangChain (for RAG pipeline), Sentence Transformers (for embeddings), FAISS (vector database for retrieval).
- **PDF Handling**: PyPDF (for loading and splitting the document).
- **LLM Integration**: Groq API (using models like Llama 3.3 70B for fast inference).
- **Development/Deployment**: Kaggle (for prototyping and testing).
- **Other**: Hugging Face models for embeddings, with secrets management for API keys.

### RAG Concept
Retrieval-Augmented Generation (RAG) is a technique that enhances LLMs by combining retrieval from a knowledge base with generation. It works by:
1. Indexing external data (e.g., PDF chunks) into a vector store using embeddings.
2. For a query, retrieving the most relevant chunks.
3. Augmenting the LLM prompt with these chunks to generate informed responses.
This avoids relying solely on the LLM's pre-trained knowledge, reducing errors.

### Why RAG is Useful
RAG is ideal for domain-specific apps like this medical chatbot because it ensures responses are grounded in accurate, updatable sources (e.g., the encyclopedia), minimizes hallucinations, and doesn't require expensive LLM fine-tuning. It's scalable for education, research, or quick info access, while being cost-effective with fast APIs like Groq. 

# Install Required Packages

In [1]:
!pip install -q langchain langchain-community langchain-groq pypdf faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:0

# Import Libraries and Set Up API Key

In [2]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from kaggle_secrets import UserSecretsClient

# Get Groq API key from Kaggle secrets
user_secrets = UserSecretsClient()
groq_api_key = user_secrets.get_secret("GROQ_API_KEY")  # Add this secret in your notebook

os.environ["GROQ_API_KEY"] = groq_api_key

# Load and Process the PDF

In [3]:
# Path to your uploaded PDF
pdf_path = "/kaggle/input/medical/The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf"  # Update with actual path

# Load PDF
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Loaded {len(chunks)} chunks from the PDF.")

Loaded 23444 chunks from the PDF.


# Create Embeddings and Vector Store

In [4]:
# Embeddings model (from HuggingFace/Sentence-Transformers)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create vector store
vectorstore = FAISS.from_documents(chunks, embeddings)

# Save the vector store locally (optional, for reuse)
vectorstore.save_local("faiss_index")

/tmp/ipykernel_36/3631328067.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
2025-09-20 20:08:04.711403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758398884.901838      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758398884.953966      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for pl

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Set Up the Groq LLM and RAG Chain

In [7]:
# Groq LLM (choose a model, e.g., 'llama-3.3-70b-versatile' for balance of speed/quality)
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7)

# Retriever (top 3 relevant chunks)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Stuff retrieved docs into prompt
    retriever=retriever,
    return_source_documents=True  # Optional: To see sources
)

# Test the Chatbot

In [8]:
def ask_question(query):
    result = qa_chain({"query": query})
    return result["result"]

# Example test query
test_query = "What are the symptoms of diabetes?"  # Replace with your query
response = ask_question(test_query)
print("Query:", test_query)
print("Response:", response)

Query: What are the symptoms of diabetes?
Response: According to the text, the classic symptoms of diabetes include:

1. Feeling tired and sick
2. Frequent urination
3. Excessive thirst
4. Excessive hunger
5. Weight loss

Additionally, other symptoms that may occur in Type II diabetes include:

1. Lethargy
2. Sudden weight loss
3. Slow wound healing
4. Urinary tract infections
5. Gum disease
6. Blurred vision

It's also mentioned that symptoms of Type II diabetes can begin so gradually that a person may not know that they have it.


In [9]:
while True:
    user_input = input("Ask a medical question (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    response = ask_question(user_input)
    print("Response:", response)

Ask a medical question (or type 'exit' to quit):  what are the symptoms of dengue?


Response: The symptoms of dengue can vary depending on the severity of the illness and the age of the person infected. Here are the symptoms mentioned in the context:

**Common symptoms:**

* Fever
* Headache
* Muscle and joint pain
* Loss of appetite
* Rash
* Nausea and vomiting
* Weakness
* Fatigue

**Symptoms in children:**

* Sore throat
* Runny nose
* Slight cough
* Fever lasting for a week or less

**Severe symptoms (Dengue Hemorrhagic Fever - DHF):**

* Bleeding from the mucous membranes and under the skin surface
* Dark blood in stools and vomit
* Jaundice
* Delirium
* Seizures
* Stupor
* Coma
* Hemorrhaging from the nose, gums, and injection sites
* Bleeding from the gastrointestinal, genitourinary, and respiratory tracts

**Characteristic combination of symptoms:**

* Fever, rash, and headache are called the "dengue triad"


Ask a medical question (or type 'exit' to quit):  exit 


Response: I don't know what you are referring to with "exit". The provided context does not contain any relevant information about an "exit". Can you please provide more context or clarify your question?


Ask a medical question (or type 'exit' to quit):  exit
